# ABS Quarterly International Trade 5302

Notes

The current account balance and the balance of trade are both components of a country’s balance of payments, but they are not the same. Here’s a breakdown of the differences:

⸻

Balance of Trade (BoT):
-	Definition: The difference between the value of a country’s exports and imports of goods over a certain period.
-	Formula:
	{Balance of Trade} = {Exports of Goods} - {Imports of Goods}
-	Focuses only on: Physical goods (also called merchandise trade).
-	Can be:
	-	Surplus (exports > imports)
	-	Deficit (imports > exports)

⸻

Current Account Balance:
-	Definition: A broader measure that includes the balance of trade plus other income and transfers.
-	Components:
	1.	Balance of Trade (goods)
	2.	Trade in Services (e.g. tourism, banking, transport)
	3.	Primary Income (e.g. dividends, interest, wages from abroad)
	4.	Secondary Income (e.g. foreign aid, remittances, pensions)
-	Formula (simplified):
	{Current Account} = {BoT} + {Services} + {Primary Income} + {Secondary Income}

⸻

Key Difference:
-	The balance of trade is a subset of the current account.
-	The current account gives a fuller picture of a country’s international economic position.

⸻

## Python set-up

In [1]:
# analytic imports
import pandas as pd
import readabs as ra
from readabs import metacol as mc

# local imports
from abs_helper import get_abs_data, get_gdp
from mgplot import line_plot_finalise, multi_start

# pandas display settings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

# display charts in this notebook
SHOW = False

## Get data from ABS

In [2]:
abs_dict, meta, source, _ = get_abs_data("5302.0")
plot_times = 0, (5 * -4 -1)

## Plot

### Headline charts

In [3]:
def headline_data() -> tuple[dict[str, tuple[pd.Series, str]], str]:
    """Returns a tuple. The first element of the tuple is a dictionary
    of headline data. The dictionary key is the title of the data.
    The value is a tuple of the data and the units of that data.
    The second elkement of the returned tuple is the ABS table name."""

    table = "530204"
    wanted = {
        "Current Account Balance": "Current account ;",
        "Balance of Trade in Goods and Services": "Goods and Services ;",
        "Net Primary Income": "Primary income ;",
        "Net Secondary Income": "Secondary income ;",
    }

    box: dict[str, tuple[pd.Series, str]] = {}
    stype = "Seasonally Adjusted"
    for title, handle in wanted.items():
        selector = {stype: mc.stype, handle: mc.did, table: mc.table}
        _table, series_id, units = ra.find_abs_id(meta, selector, exact_match=True)
        box[title] = (abs_dict[table][series_id], units)

    return box, table

In [4]:
def headline_charts(box: dict[str, tuple[pd.Series, str]], table: str) -> None:
    """Produce headline charts."""

    def plotme(series, title, units, **kwargs):
        rfooter = f"{source} {table} {kwargs.get('rfooter', '')}"
        multi_start(
            series,
            function=line_plot_finalise,
            starts=plot_times,
            title=title,
            ylabel=units,
            width=2,
            y0=True,
            rfooter=rfooter,
            lfooter="Australia. Current Prices. ",
            show=SHOW,
        )

    # --- Note GDP figures come out a day to a week after the BOP figures
    gdp, _gdp_units = get_gdp("CP", "SA")
    for title, (series, units) in box.items():
        recal_series, recal_units = ra.recalibrate(series, units)
        plotme(recal_series, title, recal_units)
        plotme(
            series / gdp * 100, title + " as a % of GDP", "Per cent", rfooter="5206.0"
        )

    net_income = box["Net Primary Income"][0] + box["Net Secondary Income"][0]
    recal_net_income, recal_net_income_units = ra.recalibrate(
        net_income, box["Net Primary Income"][1]
    )
    plotme(recal_net_income, "Net Income", recal_net_income_units)
    plotme(
        net_income / gdp * 100, "Net Income as % of GDP", "Per cent", rfooter="5206.0"
    )

In [5]:
headline_charts(*headline_data())

## Finished

In [6]:
# watermark
%load_ext watermark
%watermark --python --conda --machine --iversions --watermark

Python implementation: CPython
Python version       : 3.13.3
IPython version      : 9.3.0

conda environment: n/a

Compiler    : Clang 20.1.0 
OS          : Darwin
Release     : 24.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

mgplot : 0.1.6
readabs: 0.0.30
pandas : 2.2.3

Watermark: 2.5.0



In [7]:
print("Finished")

Finished
